# Tratando os dados das empresas da cvm e salvando em uma tabela no Sql Lite

In [1]:
#!pip install pyarrow

In [2]:
import pandas as pd

import config as cfg
import helpers as hlp

import pyarrow as pa
import pyarrow.parquet as pq

In [3]:
df_empresas = pd.read_csv(cfg.ARQUIVO_DADOS_EMPRESA_CVM)

df_empresas.head()

,CNPJ,NOME,TIPO DE PARTICIPANTE,CÓDIGO CVM,SITUAÇÃO REGISTRO
0,31.694.680/0001-14,A C M AGRICOLA SA,CIAS INCENTIVADAS REGISTRADAS,505480,Cancelado em 19/11/2008
1,08.690.448/0001-96,A G HOTEIS E TURISMO SA,CIAS INCENTIVADAS REGISTRADAS,502316,Cancelado em 24/08/2007
2,92.659.614/0001-06,A J RENNER SA IND E PARTIP,CIAS ABERTAS,35,Cancelado em 17/06/1998
3,04.895.165/0001-20,A M FIDALGO SA MAT DE CONSTR,CIAS INCENTIVADAS REGISTRADAS,504076,Cancelado em 28/09/2008
4,06.932.909/0001-64,A O GASPAR INDS SA,CIAS INCENTIVADAS REGISTRADAS,503592,Cancelado em 14/02/2012


In [4]:
# Removendo acentos e caracteres especiais das colunas.

df_empresas.rename(columns=lambda x: hlp.remover_acentuacao(x), inplace=True)

df_empresas.head()

,CNPJ,NOME,TIPO DE PARTICIPANTE,CODIGO CVM,SITUACAO REGISTRO
0,31.694.680/0001-14,A C M AGRICOLA SA,CIAS INCENTIVADAS REGISTRADAS,505480,Cancelado em 19/11/2008
1,08.690.448/0001-96,A G HOTEIS E TURISMO SA,CIAS INCENTIVADAS REGISTRADAS,502316,Cancelado em 24/08/2007
2,92.659.614/0001-06,A J RENNER SA IND E PARTIP,CIAS ABERTAS,35,Cancelado em 17/06/1998
3,04.895.165/0001-20,A M FIDALGO SA MAT DE CONSTR,CIAS INCENTIVADAS REGISTRADAS,504076,Cancelado em 28/09/2008
4,06.932.909/0001-64,A O GASPAR INDS SA,CIAS INCENTIVADAS REGISTRADAS,503592,Cancelado em 14/02/2012


In [5]:
# Separand o campo 'SITUACAO REGISTRO' em outros dois.
df_empresas[['SITUACAO','DATA_ATUALIZACAO']] = df_empresas['SITUACAO REGISTRO'].str.split(" em ",expand=True)

df_empresas.head()

,CNPJ,NOME,TIPO DE PARTICIPANTE,CODIGO CVM,SITUACAO REGISTRO,SITUACAO,DATA_ATUALIZACAO
0,31.694.680/0001-14,A C M AGRICOLA SA,CIAS INCENTIVADAS REGISTRADAS,505480,Cancelado em 19/11/2008,Cancelado,19/11/2008
1,08.690.448/0001-96,A G HOTEIS E TURISMO SA,CIAS INCENTIVADAS REGISTRADAS,502316,Cancelado em 24/08/2007,Cancelado,24/08/2007
2,92.659.614/0001-06,A J RENNER SA IND E PARTIP,CIAS ABERTAS,35,Cancelado em 17/06/1998,Cancelado,17/06/1998
3,04.895.165/0001-20,A M FIDALGO SA MAT DE CONSTR,CIAS INCENTIVADAS REGISTRADAS,504076,Cancelado em 28/09/2008,Cancelado,28/09/2008
4,06.932.909/0001-64,A O GASPAR INDS SA,CIAS INCENTIVADAS REGISTRADAS,503592,Cancelado em 14/02/2012,Cancelado,14/02/2012


In [6]:
# Renomeando, reordenando as colunas e iniciando o index em 1

df_empresas = df_empresas[['CODIGO CVM', 'NOME', 'TIPO DE PARTICIPANTE', 'CNPJ', 'SITUACAO','DATA_ATUALIZACAO']]

df_empresas.columns = ['CODIGO_CVM', 'NOME', 'TIPO_PARTICIPANTE', 'CNPJ', 'SITUACAO','DATA_ATUALIZACAO']

df_empresas.index = df_empresas.reset_index().index + 1

df_empresas.head()

,CODIGO_CVM,NOME,TIPO_PARTICIPANTE,CNPJ,SITUACAO,DATA_ATUALIZACAO
1,505480,A C M AGRICOLA SA,CIAS INCENTIVADAS REGISTRADAS,31.694.680/0001-14,Cancelado,19/11/2008
2,502316,A G HOTEIS E TURISMO SA,CIAS INCENTIVADAS REGISTRADAS,08.690.448/0001-96,Cancelado,24/08/2007
3,35,A J RENNER SA IND E PARTIP,CIAS ABERTAS,92.659.614/0001-06,Cancelado,17/06/1998
4,504076,A M FIDALGO SA MAT DE CONSTR,CIAS INCENTIVADAS REGISTRADAS,04.895.165/0001-20,Cancelado,28/09/2008
5,503592,A O GASPAR INDS SA,CIAS INCENTIVADAS REGISTRADAS,06.932.909/0001-64,Cancelado,14/02/2012


In [7]:
#Convertendo DATA_ATUALIZACAO em data.
df_empresas['DATA_ATUALIZACAO'] = pd.to_datetime(df_empresas['DATA_ATUALIZACAO'], format='%d/%m/%Y')

df_empresas.head()

,CODIGO_CVM,NOME,TIPO_PARTICIPANTE,CNPJ,SITUACAO,DATA_ATUALIZACAO
1,505480,A C M AGRICOLA SA,CIAS INCENTIVADAS REGISTRADAS,31.694.680/0001-14,Cancelado,2008-11-19
2,502316,A G HOTEIS E TURISMO SA,CIAS INCENTIVADAS REGISTRADAS,08.690.448/0001-96,Cancelado,2007-08-24
3,35,A J RENNER SA IND E PARTIP,CIAS ABERTAS,92.659.614/0001-06,Cancelado,1998-06-17
4,504076,A M FIDALGO SA MAT DE CONSTR,CIAS INCENTIVADAS REGISTRADAS,04.895.165/0001-20,Cancelado,2008-09-28
5,503592,A O GASPAR INDS SA,CIAS INCENTIVADAS REGISTRADAS,06.932.909/0001-64,Cancelado,2012-02-14


In [8]:
hlp.verificar_dados_faltando(df_empresas)

Series([], dtype: int64)

In [9]:
table_empresas = pa.Table.from_pandas(df_empresas)

hlp.ensure_folder_exists(cfg.PARQUET_DADOS_EMPRESA_CVM)

# Write the PyArrow table to a Parquet file
pq.write_table(table_empresas, cfg.PARQUET_DADOS_EMPRESA_CVM)

In [11]:
df_empresas2 = pd.read_csv(cfg.ARQUIVO_DADOS_EMPRESA_CVM)

df_empresas2.to_parquet(cfg.PARQUET_DADOS_EMPRESA_CVM2)